
# Environment Setup for DeepLabCut Evaluation

## Introduction
This notebook is dedicated to evaluating the inference performance of DeepLabCut in terms of both tracking quality and latency. Below are the steps to set up the necessary environment on Ubuntu 22.04.

## Installation

### Create Conda Environment
To begin, create a Conda environment named `DLC` with the necessary packages. Run the following commands in your terminal:

```bash
conda create -n DLC python==3.10
conda activate DLC
pip install deeplabcut==3.0.0rc7
```

### Additional Libraries
After setting up the Conda environment, install the following Python libraries using pip:

- **tqdm** for progress bars:
  ```bash
  pip install tqdm==4.66.4
  ```
- **tabulate** for table creation and formatting:
  ```bash
  pip install tabulate==0.9.0
  ```
- **motmetrics** for computing metrics for object tracking:
  ```bash
  pip install motmetrics==1.4.0
  ```
- **mmengine** for the registry :
  ```bash
  pip install mmengine-lite
  ```


In [4]:
%%capture
import os
import os.path
import pandas as pd
import os
import pandas as pd
from mmengine.config import Config
import cv2
from collections import defaultdict

import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from utils.downsize_video import downsize_video
from utils.dlc300_inference_w_timer import analyze_videos

### Load the DLC config

You need to make sure you have a valid DLC checkpoint trained on MICE. For more informations, please visit DLC's documentation: https://deeplabcut.github.io/DeepLabCut/docs/quick-start/tutorial_maDLC.html

In [ ]:
cfg_path = "../../configs/deeplabcut/evaluation.py"
cfg = Config.fromfile(cfg_path)

## Latency Comparison Standards

For a fair latency comparison, it is crucial that all algorithms perform inference on images of the same resolution. This standardization ensures that any observed differences in processing speed are due to the algorithms themselves, not variations in image size or quality.

For this reason, we need to format the video so its resolution matches those used for the PrecisionTrack and SLEAP evaluations.

In [21]:
videos = []
for video in cfg.videos:
    ori_video = cv2.VideoCapture(video)
    ori_width = int(ori_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    ori_height = int(ori_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if ori_height != cfg.img_size[0] or ori_width != cfg.img_size[1]:
        print(f"Rescaling video from {ori_height}x{ori_width} to {cfg.img_size[0]}x{cfg.img_size[1]}")
        split = os.path.splitext(video)
        rescaled_video_path = f"{split[0]}_{cfg.img_size[0]}x{cfg.img_size[1]}{split[1]}"
        video = downsize_video(
            video,
            rescaled_video_path,
            height=cfg.img_size[0],
            width=cfg.img_size[1],
        )
    videos.append(video)

Rescaling video from 1536x1536 to 640x640


In [28]:
video = rescaled_video_path

path_to_dlc_cfg = cfg.config
batch_size = cfg.batch_size
n_tracks = cfg.n_tracks
save_as_csv = cfg.save_as_csv
shuffle = 1

fps = analyze_videos(
    path_to_dlc_cfg, videos=[video], shuffle=shuffle, gputouse=0, batchsize=batch_size, n_tracks=n_tracks, save_as_csv=save_as_csv, auto_track=True
)[1][0]
print(f"{video} analysis ran at {fps:.3f} FPS.")

Using snapshot-30000 for model /home/vincent/Documents/precision_track/precision_track/evaluation/comparisons/ckpts/deeplabcut/dlc-models/iteration-0/miceNov03-trainset80shuffle1


/home/vincent/mambaforge/envs/DLC/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Activating extracting of PAFs


2025-06-12 08:55:35.632243: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-12 08:55:35.633608: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-12 08:55:35.634767: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Starting to analyze %  ../../assets/20mice_640x640.avi
Loading  ../../assets/20mice_640x640.avi
Duration of video [s]:  52.54 , recorded with  28.0 fps!
Overall # of frames:  1471  found with (before cropping) frame dimensions:  640 640
Starting to extract posture from the video(s) with batchsize: 30


...

Video Analyzed. Saving results in ../../assets...
Using snapshot-30000 for model /home/vincent/Documents/precision_track/precision_track/evaluation/comparisons/ckpts/deeplabcut/dlc-models/iteration-0/miceNov03-trainset80shuffle1
Processing...  ../../assets/20mice_640x640.avi
Analyzing ../../assets/20mice_640x640DLC_dlcrnetms5_miceNov03shuffle1_30000.h5


100%|██████████| 1471/1471 [00:03<00:00, 376.81it/s]
1471it [00:02, 529.43it/s]


The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.
Processing...  ../../assets/20mice_640x640.avi


100%|██████████| 282/282 [00:00<00:00, 484.25it/s]
/home/vincent/mambaforge/envs/DLC/lib/python3.10/site-packages/deeplabcut/refine_training_dataset/stitch.py:939: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  df.to_hdf(output_name, "tracks", format="table", mode="w")


SAVING TRACKING RESULTS TO: ../../assets/20mice_640x640DLC_dlcrnetms5_miceNov03shuffle1_30000_el.h5

| FPS    |   Inference Time (s) |
|--------|----------------------|
| Mean   |               42.089 |
| Std    |                0.000 |
| Min    |               42.089 |
| Median |               42.089 |
| Max    |               42.089 |
../../assets/20mice_640x640.avi analysis ran at 42.089 FPS.
